In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import scipy.io as sio
import csv
import pylab as plt
import random

In [3]:
def save_prediction(pred, filename):
    """
    Save the output of the regression as csv file
    """
    pred=pred.flatten()
    totalRows = pred.shape[0] 
    filename = filename+'.csv'
    with open(filename, mode = 'w') as result:
        csv_writer = csv.writer(result, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(['dataid', 'prediction'])
        
        for i in range(totalRows):
             csv_writer.writerow([str(i+1), pred[i]])

In [4]:
mat_contents = sio.loadmat(file_name='./MSdata.mat')
#mat_contents

In [5]:
test_x = mat_contents['testx']
train_x = mat_contents['trainx']
train_y = mat_contents['trainy']

In [6]:
max_train_x = np.amax(abs(train_x), axis=0)
norm_train_x = np.divide(train_x, max_train_x)

In [19]:
# Cut off features which are not meaningful
std_features = np.std(norm_train_x, axis=0)
index = np.where(std_features > 0.032)
traincut_x = norm_train_x[:, index[0]]

In [20]:
traincut_x.shape

(463715, 51)

In [12]:
# SVM Regression
from sklearn import svm
svm = svm.SVR(kernel='rbf', cache_size=2000, C=1)
svm.fit(traincut_x, train_y.flatten())

SVR(C=1, cache_size=2000, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [13]:
testcut_x = np.divide(test_x, max_train_x)[:, index[0]]
svm_pred_y = svm.predict(testcut_x)

In [14]:
save_prediction(svm_pred_y, 'svm_rbf_trainall_feat0p032_result')

In [ ]:
# OLS (there are no penalty terms)
reg = linear_model.LinearRegression()
reg.fit(train_x, train_y)

In [ ]:
ols_pred_y = reg.predict(test_x)

In [ ]:
# Ridge Regression
ridge = linear_model.Ridge(alpha=.5)
ridge.fit(norm_train_x, train_y)

In [ ]:
ridge_pred_y = ridge.predict(np.divide(test_x, max_train_x))

In [ ]:
# Lasso Regression
lasso = linear_model.Lasso(alpha=0.2)
lasso.fit(norm_train_x, train_y)

In [ ]:
lasso_pred_y = lasso.predict(np.divide(test_x, max_train_x))

### Feature selection

In [ ]:
nsamp = 200
sample_index = random.sample(range(len(test_x)), nsamp)
subtest_x = train_x[sample_index,:]
subtest_y = train_y[sample_index]

In [ ]:
subtest_x.shape

In [ ]:
plt.hist(subtest_x[:,0])
plt.hist(subtest_x[:,10])
#plt.hist(subtest_x[:,20])

In [ ]:
plt.hist(subtest_x[:,20], label='20')
plt.hist(subtest_x[:,21], label='21')
plt.hist(subtest_x[:,22], label='22')
plt.hist(subtest_x[:,24], label='24')
plt.title('Features at 20-24 index')
plt.legend()

In [ ]:
plt.figure(figsize=(10,6))
imshow(subtest_x, vmin=0, vmax=100)

In [ ]:
low_saturated= np.where(subtest_x<0)[1]

In [ ]:
def remove_similar_feature_row(train_x, train_y, cutoff=100):
    """
    Go through test sample and remove data point with indistingishable features
    """
    std_features = np.std(train_x, axis=1)
    index = np.where(std_features>cutoff)
    X = train_x.copy()[index]
    Y = train_y.copy()[index]
    return X, Y

In [ ]:
traincut_x, traincut_y= remove_similar_feature_row(train_x, train_y)

In [ ]:
regcut = linear_model.LinearRegression()
regcut.fit(traincut_x, traincut_y)

In [ ]:
testcut_pred_y = regcut.predict(test_x)

In [ ]:
save_prediction(testcut_pred_y, 'cut_linearreg')

In [ ]:
bayes_cut = linear_model.BayesianRidge()
bayes_cut.fit(traincut_x, traincut_y)

In [ ]:
bayescut_pred_y = bayes_cut.predict(test_x)

In [ ]:
save_prediction(bayescut_pred_y, 'cut_bayes')

In [ ]:
len(np.where(low_saturated == 10)[0])

In [ ]:
plt.hist(subtest_x[:,3])
plt.hist(subtest_x[:,4])
plt.hist(subtest_x[:,5])
plt.hist(subtest_x[:,6])

In [ ]:
plt.hist(subtest_x[:,13])
plt.hist(subtest_x[:,14])
plt.hist(subtest_x[:,15])
plt.hist(subtest_x[:,16])


In [ ]:
subtest_x[:,13].shape

In [ ]:
len(np.where(saturated == 13)[0])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 

In [ ]:
train_y = train_y.astype('int32')

In [ ]:
trainx = torch.from_numpy(train_x)
trainy = torch.from_numpy(train_y)

In [ ]:
batch_size = 100

X = torch.from_numpy(train_x).double()
y = torch.from_numpy(train_y).double()

#train_data = torch.utils.data.TensorDataset(X, y)
#train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

X = Variable(X).float()
y = Variable(y).float()
#type(torch.LongTensor)
train_data = torch.utils.data.TensorDataset(X, y)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [ ]:
lin = nn.Linear(90,1)
opt = torch.optim.SGD(lin.parameters(), lr=0.1)
loss_fn = F.mse_loss
loss = loss_fn(lin(X), y)

In [ ]:
torch.nn.utils.clip_grad_norm_(lin.parameters(), 0.25)

In [ ]:
def get_batch(source, i):
    bptt=20000
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len]
    return data, target

In [ ]:
def fit(num_epochs, model, loss_fn, opt):
    # Define a utility function to train the model
    for epoch in range(num_epochs):
        for xb,yb in train_loader:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
            opt.step()
            opt.zero_grad()
    print('Training loss: ', loss_fn(model(X), y))

In [ ]:
fit(1000, lin, loss_fn, opt)

In [ ]:
testx = torch.from_numpy(test_x).double()
testX = Variable(testx).float()

In [ ]:
pred = lin(testX)

In [ ]:
pred

In [ ]:
test_pred_y

In [ ]:
def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

def train():
    # Turn on training mode which enables dropout.
    X.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(args.batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % args.log_interval == 0 and batch > 0:
            cur_loss = total_loss / args.log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt, lr,
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [ ]:
# Save the result into 'result.csv'

totalRows = test_pred_y.shape[0]
with open('result.csv', mode='w') as result_file:
    csv_writer = csv.writer(result_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
    csv_writer.writerow(['dataid','prediction'])
    
    for i in range(totalRows):
        csv_writer.writerow([str(i+1),test_pred_y[i][0]])
    